In [18]:
import  requests 
import pandas as pd
import investpy


from urllib.error import URLError, HTTPError
from urllib3.exceptions import SSLError

from requests.exceptions import HTTPError
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from socket import timeout

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import time
import sys

In [5]:
link = 'https://www.investing.com/equities/turkey'

list_of_stock_links = []

In [7]:

chrome_driver_path = 'selenium_p/chromedriver'
driver = webdriver.Chrome(executable_path=chrome_driver_path)
driver.set_page_load_timeout('10')
driver.get(link)


driver.find_element_by_id('19158').click()
time.sleep(5)

html = driver.page_source

time.sleep(4)
driver.quit()



In [8]:
list_of_stock_links = []
soup = BeautifulSoup(html, "html.parser")
    
data = soup.findAll('table', {'id': 'cross_rate_markets_stocks_1'})
data2 = data[0].findAll('tbody')[0].findAll('tr')
    
for stock in data2:
    link = str(stock.findAll('a')).split('href="')[1].split('"')[0]
    list_of_stock_links.append(f"www.investing.com{link.strip()}")

print(len(list_of_stock_links))
print(list_of_stock_links[0])

30
www.investing.com/equities/kardemir-(d)


In [9]:
df = pd.DataFrame(list_of_stock_links, columns =['Links'])
df.to_csv('datasets/BIST_30_links.csv', encoding='utf-8', index=False)

df.head() 

,Links
0,www.investing.com/equities/kardemir-(d)
1,www.investing.com/equities/sise-cam
2,www.investing.com/equities/yapi-ve-kredi-bank.
3,www.investing.com/equities/petkim
4,www.investing.com/equities/emlak-konut-gmyo


In [23]:
bist30_links_stockname = []
c = 0
## Getting stocks names, full names
for url in list_of_stock_links:
    try: 

        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
        reg_url = 'https://'+url
        req = Request(url=reg_url, headers=headers) 
        html = urlopen(req).read()     
        soup = BeautifulSoup(html, "html.parser")
        data = soup.findAll('div', {'class': 'instrumentHead'})[0]
        full_name = str(data).split('"name">')[1].split('</h1>')[0].strip()
        stock_name = str(data).split('"name">')[1].split('</h1>')[0].strip().split('(')[1].split(')')[0]
        bist30_links_stockname.append([url ,full_name ,stock_name])
         
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    
print(len(bist30_links_stockname))

30


In [24]:
df2 = pd.DataFrame(bist30_links_stockname, columns =['Link','Full_Name','Stock_Name'])
df2.to_csv('datasets/BIST_30_links_stockName.csv', encoding='utf-8', index=False)

df2.head() 

,Link,Full_Name,Stock_Name
0,www.investing.com/equities/kardemir-(d),Kardemir Karabuk Demir Celik Sanayi ve Ticaret...,KRDMD
1,www.investing.com/equities/sise-cam,Turkiye Sise ve Cam Fabrikalari AS (SISE),SISE
2,www.investing.com/equities/yapi-ve-kredi-bank.,Yapi ve Kredi Bankasi AS (YKBNK),YKBNK
3,www.investing.com/equities/petkim,Petkim Petrokimya Holding AS (PETKM),PETKM
4,www.investing.com/equities/emlak-konut-gmyo,Emlak Konut Gayrimenkul Yatirim Ortakligi AS (...,EKGYO


In [25]:
df3 = df2[['Stock_Name']].copy()
df2.to_csv('datasets/BIST_30_stockNames.csv', encoding='utf-8', index=False)

df3.head()

,Stock_Name
0,KRDMD
1,SISE
2,YKBNK
3,PETKM
4,EKGYO


In [26]:
df3 = pd.read_csv("datasets/BIST_30_stockNames.csv") 
df3.head()

,Link,Full_Name,Stock_Name
0,www.investing.com/equities/kardemir-(d),Kardemir Karabuk Demir Celik Sanayi ve Ticaret...,KRDMD
1,www.investing.com/equities/sise-cam,Turkiye Sise ve Cam Fabrikalari AS (SISE),SISE
2,www.investing.com/equities/yapi-ve-kredi-bank.,Yapi ve Kredi Bankasi AS (YKBNK),YKBNK
3,www.investing.com/equities/petkim,Petkim Petrokimya Holding AS (PETKM),PETKM
4,www.investing.com/equities/emlak-konut-gmyo,Emlak Konut Gayrimenkul Yatirim Ortakligi AS (...,EKGYO


In [27]:

## getting stocks historical data using investPy

for stock in df3.Stock_Name: ## dont forget to create Bist30_data folder in your directory // too lazy to code it
#     print(stock)
    df4 = investpy.get_stock_historical_data(stock=stock,country='turkey',from_date='01/01/2010',to_date='01/08/2020')
    df4.to_csv(f'datasets/stocks_hist_data_bist30/{stock}_2010-2020.csv', encoding='utf-8')
